## Object Relation Model

In [1]:
import datetime as dt

from sqlmodel import create_engine, Field, Session, select, SQLModel, Relationship

### Single entity

In [2]:
class TaskList(SQLModel, table=True):
    """Class for managing task lists."""

    ListId: int | None = Field(primary_key=True)
    Name: str

In [16]:
# Create a (SQLAlchemy) engine
connection = create_engine("sqlite://", echo=True)
session = Session(connection)

In [ ]:
# Create tables based on the model metadata.
SQLModel.metadata.create_all(connection)

In [ ]:
# Create a customer object
my_list = TaskList(Name="House Keeping")
my_list

In [ ]:
# Add the list record and commit.
session.add(my_list)
session.commit()

In [ ]:
# Create the select query.
query = select(TaskList)

# Perform the query on the database.
result = session.execute(query)

# Fetch all results.
# Note: Results are Python / SQLModel objects!
result.fetchmany(3)


In [ ]:
my_list.Name = "New House Keeping"
session.add(my_list)
session.commit()

In [21]:
# Delete the engine / close connection
connection.dispose(close=True)

### Foreign Keys

In [ ]:
class TaskList(SQLModel, table=True):
    """Class for managing task lists."""

    list_id: int | None = Field(primary_key=True)
    name: str


class Task(SQLModel, table=True):
    """Class for managing tasks."""

    # Task related fields
    task_id: int | None = Field(primary_key=True)
    description: str
    state: str | None = "New"
    updated: dt.datetime | None = dt.datetime.now()

    # Foreign key to TaskList
    list_id: int = Field(foreign_key="tasklist.list_id")



In [ ]:
# Start engine and create the data model.
engine = create_engine("sqlite:///", echo=True)
SQLModel.metadata.create_all(engine)

In [ ]:
# Create a session.
session = Session(engine)

In [ ]:
# Define a new task list.
my_list = TaskList(name="House Keeping")
my_list

In [ ]:
# Define a new task.
first_task = Task(description="Walk the dog")
first_task

In [ ]:
# Add the address first to generate an ID.
session.add(my_list)

# Flush the session to get the generated ID.
session.flush()

print("Generated Address ID: ", my_list.list_id)

In [11]:
# Link the address to the customer.
first_task.list_id = my_list.list_id

# Now store the customer.
session.add(first_task)

In [ ]:
# Commit the changes.
session.commit()

In [ ]:
# Read linked tables using where().
query =  select(TaskList, Task).join(Task)
results = session.exec(query)

# Print the results
for result in results.fetchall():
    tasklist, task = result
    print("-" * 80)
    print ("TaskList: ", tasklist)
    print ("Task:     ", task)


In [ ]:
# Delete the engine / close connection
session.close()
engine.dispose(close=True)

### Relationships

In [2]:
class TaskList(SQLModel, table=True):
    """Class for managing task lists."""

    list_id: int | None = Field(primary_key=True)
    name: str

    # Define Task relationship as a property.
    tasks: list["Task"] = Relationship(back_populates="list")


class Task(SQLModel, table=True):
    """Class for managing tasks."""

    # Task related fields
    task_id: int | None = Field(primary_key=True)
    description: str
    state: str | None = "New"
    updated: dt.datetime | None = dt.datetime.now()

    # Foreign key to TaskList
    list_id: int = Field(foreign_key="tasklist.list_id")

    # Define list relationship as a  property.
    list: TaskList = Relationship(back_populates="tasks")

In [ ]:
# Start engine and create the data model.
engine = create_engine("sqlite:///", echo=True)
SQLModel.metadata.create_all(engine)

In [4]:
# Create session
session = Session(engine)

In [5]:
# Define a Task.
first_task = Task(description="Walk the dog")

In [6]:
# Define the TaskList.
# Note: The Task object is passed along using the relationship!
my_list = TaskList(name="House Keeping", tasks=[first_task])

In [ ]:
# Store the customer
# Note: The address is stored automatically!
session.add(my_list)
session.commit()

In [ ]:
query = select(TaskList)
task_lists = session.exec(query).fetchall()
task_lists

In [ ]:
# Get the address via the relationship
task_lists[0].tasks

In [ ]:
# Access address properties directly.
task_lists[0].tasks[0].description

In [ ]:
# Delete the engine / close connection
session.close()
engine.dispose(close=True)